In [ ]:
import re
import string
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import  CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/nlp-getting-started/train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.drop(['id','keyword','location'], axis=1, inplace=True)

In [ ]:
train_df.isnull().sum()

In [ ]:
counts = train_df['target'].value_counts()
sns.barplot(x=counts.index, y=counts)
plt.xlabel('Target')
plt.ylabel('Count')

In [ ]:
train_df['Text_length'] = train_df['text'].apply(len)

In [ ]:
length_text = sns.FacetGrid(data=train_df, col='target')
length_text.map(plt.hist, 'Text_length', bins=20, color='r')

In [ ]:
def plot_(a):
    x1 = train_df[train_df.target == 1][a]
    x2 = train_df[train_df.target == 0][a]
    plt.subplot(1,1,1)
    __= plt.hist(x2, alpha=0.5, color="grey", bins=50)
    _ = plt.hist(x1, alpha=0.8, color="red", bins=50)
    return __

In [ ]:
_ = plot_("Text_length")

In [ ]:
train_df['words_counts'] = train_df.text.str.split().map(lambda x: len(x))

In [ ]:
train_df.head()

In [ ]:
_ = plot_("words_counts")

In [ ]:
train_df['punctuation_count'] = train_df['text'].map(lambda x: len([c for c in str(x) if c in string.punctuation]))

In [ ]:
train_df.head()

In [ ]:
_ =plot_('punctuation_count')

In [ ]:
def clean_text(text):
    text = text.lower()                                  # lower-case all characters
    text =  re.sub(r'@\S+', '',text)                     # remove twitter handles
    text =  re.sub(r'http\S+', '',text)                  # remove urls
    text =  re.sub(r'pic.\S+', '',text) 
    text =  re.sub(r"[^a-zA-Z+']", ' ',text)             # only keeps characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text+' ')      # keep words with length>1 only
    text = "".join([i for i in text if i not in string.punctuation])
    words = nltk.tokenize.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')   # remove stopwords
    text = " ".join([i for i in words if i not in stopwords and len(i)>2])
    text= re.sub("\s[\s]+", " ",text).strip()            # remove repeated/leading/trailing spaces
    return text

In [ ]:
train_df['Text_cleaning'] = train_df.text.apply(clean_text)

In [ ]:
train_df.head()

In [ ]:
x_1 = train_df[train_df.target == 1]["Text_cleaning"]
x_0 = train_df[train_df.target == 0]["Text_cleaning"]
w_cloud_0 = WordCloud(background_color="white", max_font_size=80).generate(" ".join(x_0))
w_cloud_1 = WordCloud(background_color="white", max_font_size=80).generate(" ".join(x_1))
plt.figure(1,figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.imshow(w_cloud_0)
plt.axis("off")
plt.subplot(1, 2, 2)
plt.imshow(w_cloud_1)
plt.axis("off")

In [ ]:
X = train_df['Text_cleaning']
y = train_df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

In [ ]:
text = train_df['Text_cleaning']
counter = counter_word(text)

In [ ]:
train_df.describe()

In [ ]:
len(counter)

In [ ]:
num_words = len(counter)
max_len = 157

In [ ]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(X_train,maxlen=max_len, truncating="post")

In [ ]:
X_test = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(X_test,maxlen=max_len, truncating="post")

In [ ]:
my_callbacks  = [EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=5,
                              mode='auto')]

In [ ]:
model = Sequential()
model.add(Embedding(num_words, 32,input_length=max_len))
model.add(LSTM(64, dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(train_padded, y_train, epochs=100, validation_data=(test_padded, y_test), callbacks=my_callbacks)

In [ ]:
results = model.evaluate(test_padded, y_test, verbose=0)
print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
y_pred = model.predict_classes(test_padded)
print(classification_report(y_test, y_pred))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val']) 